<a href="https://colab.research.google.com/github/jbenasuli/final_project/blob/livia-branch/AmazonDatabase.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.1.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad.net/graph

In [6]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [7]:
# Selected furniture data
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Furniture_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df_furniture = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df_furniture.show()
print((df_furniture.count(), len(df_furniture.columns)))

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   24509695|R3VR960AHLFKDV|B004HB5E0E|     488241329|Shoal Creek Compu...|       Furniture|          4|            0|          0|   N|                Y|... desk is very ...|This desk is very...| 2015-08-31|
|         US|   34731776|R16LGVMFKIUT0G|B0042TNMMS|     205864445|Dorel Home Produc...|       Furniture|          5|    

In [8]:
# Selected personal care applianaces
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Personal_Care_Appliances_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df_personal_care = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
print((df_personal_care.count(), len(df_personal_care.columns)))

(878094, 15)


In [9]:
# Selected office products
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Office_Products_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df_office = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
print((df_office.count(), len(df_office.columns)))

(3520528, 15)


In [10]:
# Selected apparel products
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Apparel_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df_apparel = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
print((df_apparel.count(), len(df_apparel.columns)))

(9426861, 15)


In [11]:
# Selected home products
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Home_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df_home = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
print((df_home.count(), len(df_home.columns)))

(15648420, 15)


In [17]:
# Merge databases
from functools import reduce  # For Python 3.x
from pyspark.sql import DataFrame

def unionAll(*dfs):
    return reduce(DataFrame.unionAll, dfs)

df_merge = unionAll(df_furniture, df_personal_care, df_office, df_apparel, df_home)
df_merge.show()

print((df_merge.count(), len(df_merge.columns)))

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   24509695|R3VR960AHLFKDV|B004HB5E0E|     488241329|Shoal Creek Compu...|       Furniture|          4|            0|          0|   N|                Y|... desk is very ...|This desk is very...| 2015-08-31|
|         US|   34731776|R16LGVMFKIUT0G|B0042TNMMS|     205864445|Dorel Home Produc...|       Furniture|          5|    

In [18]:
# Only considering verified purchases
df_merge = df_merge.filter(df_merge.verified_purchase == "Y")

In [19]:
# Drop the non-beneficial columns
columns_to_drop = ["marketplace","review_id","product_parent","star_rating","helpful_votes","total_votes","vine","review_headline","review_body","verified_purchase"]

df_merge = df_merge.drop(*columns_to_drop)
df_merge.show()

print((df_merge.count(), len(df_merge.columns)))

+-----------+----------+--------------------+----------------+-----------+
|customer_id|product_id|       product_title|product_category|review_date|
+-----------+----------+--------------------+----------------+-----------+
|   24509695|B004HB5E0E|Shoal Creek Compu...|       Furniture| 2015-08-31|
|   34731776|B0042TNMMS|Dorel Home Produc...|       Furniture| 2015-08-31|
|    1272331|B0030MPBZ4|Bathroom Vanity T...|       Furniture| 2015-08-31|
|   45284262|B005G02ESA|Sleep Master Ulti...|       Furniture| 2015-08-31|
|   18311821|B00AVUQQGQ|Serta Bonded Leat...|       Furniture| 2015-08-31|
|   42943632|B00CFY20GQ|Prepac Shoe Stora...|       Furniture| 2015-08-31|
|   43157304|B00FKC48QA|HomCom PU Leather...|       Furniture| 2015-08-31|
|   51918480|B00N9IAL9K|  Folding Step Stool|       Furniture| 2015-08-31|
|   14522766|B001T4XU1C|Ace Bayou Adult V...|       Furniture| 2015-08-31|
|   43054112|B002HRFLBC|4D Concepts Audio...|       Furniture| 2015-08-31|
|   26622950|B006MISZOC|Z

In [20]:
# Get datatypes
df_merge.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- review_date: string (nullable = true)



In [21]:
# Convert date from string to datetime
from pyspark.sql.types import *
df_merge = df_merge.withColumn("review_date", df_merge["review_date"].cast(DateType()))

df_merge.printSchema()
df_merge.show()

root
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- review_date: date (nullable = true)

+-----------+----------+--------------------+----------------+-----------+
|customer_id|product_id|       product_title|product_category|review_date|
+-----------+----------+--------------------+----------------+-----------+
|   24509695|B004HB5E0E|Shoal Creek Compu...|       Furniture| 2015-08-31|
|   34731776|B0042TNMMS|Dorel Home Produc...|       Furniture| 2015-08-31|
|    1272331|B0030MPBZ4|Bathroom Vanity T...|       Furniture| 2015-08-31|
|   45284262|B005G02ESA|Sleep Master Ulti...|       Furniture| 2015-08-31|
|   18311821|B00AVUQQGQ|Serta Bonded Leat...|       Furniture| 2015-08-31|
|   42943632|B00CFY20GQ|Prepac Shoe Stora...|       Furniture| 2015-08-31|
|   43157304|B00FKC48QA|HomCom PU Leather...|       Furniture| 2015-08-31|
|   51918480|B00N9IAL9K|

In [22]:
from pyspark.sql.functions import year
from pyspark.sql.functions import to_date
 
df_merge = df_merge.withColumn("review_year",year(df_merge.review_date))
df_merge.show()

+-----------+----------+--------------------+----------------+-----------+-----------+
|customer_id|product_id|       product_title|product_category|review_date|review_year|
+-----------+----------+--------------------+----------------+-----------+-----------+
|   24509695|B004HB5E0E|Shoal Creek Compu...|       Furniture| 2015-08-31|       2015|
|   34731776|B0042TNMMS|Dorel Home Produc...|       Furniture| 2015-08-31|       2015|
|    1272331|B0030MPBZ4|Bathroom Vanity T...|       Furniture| 2015-08-31|       2015|
|   45284262|B005G02ESA|Sleep Master Ulti...|       Furniture| 2015-08-31|       2015|
|   18311821|B00AVUQQGQ|Serta Bonded Leat...|       Furniture| 2015-08-31|       2015|
|   42943632|B00CFY20GQ|Prepac Shoe Stora...|       Furniture| 2015-08-31|       2015|
|   43157304|B00FKC48QA|HomCom PU Leather...|       Furniture| 2015-08-31|       2015|
|   51918480|B00N9IAL9K|  Folding Step Stool|       Furniture| 2015-08-31|       2015|
|   14522766|B001T4XU1C|Ace Bayou Adult V..

In [23]:
# Create a table to count data rows per year
row_year_table = df_merge.groupby("review_year").agg({"review_year":"count"}).withColumnRenamed("count(review_year)", "year_count")
row_year_table.sort(row_year_table.review_year.desc()).show(truncate=False)

+-----------+----------+
|review_year|year_count|
+-----------+----------+
|2015       |10159789  |
|2014       |8613535   |
|2013       |4717096   |
|2012       |1616030   |
|2011       |808013    |
|2010       |442625    |
|2009       |198524    |
|2008       |103916    |
|2007       |77769     |
|2006       |20737     |
|2005       |10626     |
|2004       |5216      |
|2003       |5299      |
|2002       |3468      |
|2001       |2736      |
|2000       |1462      |
|1999       |123       |
|null       |0         |
+-----------+----------+



In [24]:
# Only considering 2010 data for week 1
df_merge = df_merge.filter(df_merge.review_year == "2010")
print((df_merge.count(), len(df_merge.columns)))

(442625, 6)


In [25]:
df_merge.toPandas().to_csv('df_merge.csv')

In [52]:
# Convert Pyspark dataframe into panda df
df_week1 = df_merge.toPandas()

In [ ]:
# Create table with customer_IDs and products bought (as  row)
# This method didn't work, there's a limit of 10,000 distinct values in pyspark

# summary_table = df_merge.groupby("customer_id").pivot("product_id").count()
# summary_table.show()

In [54]:
import pandas as pd

In [4]:
# apriori_df = (df_week1.groupby(['customer_id', 'product_id']).count().unstack().reset_index().fillna(0).set_index('customer_id'))
# apriori_df 

NameError: ignored

In [ ]:
# Summarize
summary_table = df_merge.groupby("customer_id","product_id").count()
summary_table.show()

In [26]:
product_table = df_merge.groupby("product_id").agg({"customer_id":"count"}).withColumnRenamed("count(customer_id)", "count_customers")
product_table.show()

+----------+---------------+
|product_id|count_customers|
+----------+---------------+
|B00LN17FPI|             95|
|B006H3IAYM|             15|
|B00H4SSFQW|             70|
|B0043JR5NO|            315|
|B0051CMYCO|             85|
|B00GEXJH4W|            784|
|B0103KTOPS|              5|
|B00FRR0BU0|             25|
|B006Z8OID6|              5|
|B00XGUFUGK|             10|
|B005G6Z4OK|             50|
|B00HVW3TXA|            200|
|B0017RAPTQ|            120|
|B001BX5A78|             10|
|B0016L8BD0|             35|
|B00LCIJUQY|              5|
|B004D7X2Y2|             35|
|B007T0LZF8|             35|
|B00BUHWG0O|             30|
|B00570FQRU|             15|
+----------+---------------+
only showing top 20 rows



In [32]:
product_table.agg({"count_customers": 'max'}).show()
product_table.agg({"count_customers": 'min'}).show()
product_table.agg({"count_customers": 'avg'}).show()

+--------------------+
|max(count_customers)|
+--------------------+
|               13380|
+--------------------+

+--------------------+
|min(count_customers)|
+--------------------+
|                   1|
+--------------------+

+--------------------+
|avg(count_customers)|
+--------------------+
|   5.712677669727584|
+--------------------+



In [38]:
# Random sample
# spark = SparkSession.builder.master("local[1]").appName("SparkByExamples.com").getOrCreate()

# df_merge =spark.range(1000000)
# print(df_merge.sample(0.1).collect())

[Row(id=2), Row(id=17), Row(id=20), Row(id=35), Row(id=36), Row(id=48), Row(id=50), Row(id=53), Row(id=63), Row(id=64), Row(id=65), Row(id=67), Row(id=78), Row(id=91), Row(id=94), Row(id=100), Row(id=101), Row(id=119), Row(id=136), Row(id=151), Row(id=154), Row(id=164), Row(id=169), Row(id=172), Row(id=179), Row(id=184), Row(id=189), Row(id=191), Row(id=194), Row(id=206), Row(id=213), Row(id=242), Row(id=253), Row(id=264), Row(id=268), Row(id=272), Row(id=288), Row(id=297), Row(id=304), Row(id=314), Row(id=325), Row(id=328), Row(id=373), Row(id=374), Row(id=379), Row(id=380), Row(id=394), Row(id=412), Row(id=419), Row(id=436), Row(id=441), Row(id=443), Row(id=446), Row(id=450), Row(id=452), Row(id=457), Row(id=463), Row(id=467), Row(id=476), Row(id=490), Row(id=491), Row(id=493), Row(id=513), Row(id=523), Row(id=534), Row(id=552), Row(id=553), Row(id=557), Row(id=558), Row(id=560), Row(id=572), Row(id=587), Row(id=600), Row(id=605), Row(id=609), Row(id=616), Row(id=638), Row(id=644), R

In [ ]:
# Remove products with lower count

df_merge = df.filter(df.total_votes >= 20)
df_votes.show()

In [ ]:
# Create table with customer_IDs and products bought
summary_table = df_merge.groupby("customer_id").pivot("product_id").count()
summary_table.show()